# Implementing a Component

## What Should Be Implemented as a Component?

The first step is to ensure that what you want to implement is actually a component.
Most components are composed of several blocks, with a mostly sequential forward pass.
They are intended to be used as parts of a model, and are not models themselves.

A component should have the flexibility in the input arguments to the constructor to
define the architecture of the component, including the number of layers, the number of units in each layer, and some important hyperparameters.

Examples of components are `ConvolutionalNeuralNetwork` and `MultiLayerPerceptron`.

## Implementing a Component

Here you'll see the steps you should follow to implement a component in deeplay. As an example, you'll implement the `ConvolutionalNeuralNetwork`component.

### 1. Create a New File

The first step is to create a new file in the `deeplay/components` directory. It
can be in a deeper subdirectory if it makes sense.

**The base class.**
Components generally don't have a specific base class. Sometimes it makes sense to subclass an existing component, but it is not necessary. If not, use `DeeplayModule` as the base class.

This example implements the `ConvolutionalNeuralNetwork` component.

In [1]:
from deeplay.blocks import Conv2dBlock
from deeplay.list import Sequential
from deeplay.module import DeeplayModule
import torch.nn as nn

class ConvolutionalNeuralNetwork(DeeplayModule):
    def __init__(
        self, 
        in_channels,
        hidden_channels,
        out_channels,
        out_activation=nn.ReLU,
    ):
        super().__init__()

        blocks = Sequential[Conv2dBlock]()
        for in_ch, out_ch in zip([in_channels] + hidden_channels, 
                                 hidden_channels + [out_channels]):
            block = Conv2dBlock(in_ch, out_ch, kernel_size=3, padding=0, 
                                activation=nn.ReLU)
            blocks.append(block)
        
        # Set the activation function of the last block.
        blocks[-1].activated(out_activation)

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

### 2. Add Annotations

It is important to add annotations to the class and methods to ensure that the
user knows what to expect. This is also useful for the IDE to provide 
autocomplete.

In [2]:
from typing import List, Type
from deeplay.list import Sequential
import torch

class ConvolutionalNeuralNetwork(DeeplayModule):

    # Arguments.
    in_channels: int
    hidden_channels: List[int]
    out_channels: int
    out_activation: Type[nn.Module]

    # Attributes.
    blocks: Sequential[Conv2dBlock]

    def __init__(
        self, 
        in_channels: int,
        hidden_channels: List[int],
        out_channels: int,
        out_activation: Type[nn.Module] = nn.ReLU,
    ) -> None:  
        super().__init__()

        blocks = Sequential[Conv2dBlock]()
        for in_ch, out_ch in zip([in_channels] + hidden_channels, 
                                 hidden_channels + [out_channels]):
            block = Conv2dBlock(in_ch, out_ch, kernel_size=3, padding=0, 
                                activation=nn.ReLU)
            blocks.append(block)
        
        # Set the activation function of the last block.
        blocks[-1].activated(out_activation)

    def forward(
        self, 
        x: torch.Tensor,
    ) -> torch.Tensor: 
        for block in self.blocks:
            x = block(x)
        return x

### 3. Document the Component

The next step is to document the component. This should include a description of 
the component, the input and output shapes, and the arguments that can be passed to
the component.

In [3]:
class ConvolutionalNeuralNetwork(DeeplayModule):
    """A fully convolutional neural network.

    Parameters
    ----------
    in_channels : int
        The number of input channels.
    hidden_channels : List[int]
        The number of hidden channels.
    out_channels : int
        The number of output channels.
    out_activation : Type[nn.Module]
        The type of activation function of the output layer.
    
    Attributes
    ----------
    blocks : Sequential[Conv2dBlock]
        The list of convolutional blocks.
    
    Input
    -----
    x : torch.Tensor
        The input tensor of shape (N, C, H, W).
        Where N is the batch size, C is the number of channels, H is the 
        height, and W is the width.
        Additial dimensions before C are allowed.
    
    Output
    ------
    y : torch.Tensor
        The output tensor of shape (N, out_channels, H', W').
        Where N is the batch size, out_channels is the number of output 
        channels, H is the height, and W is the width.
        Additial dimensions before out_channels will be preserved.

    Evaluation
    ----------
    ```python
    for block in blocks:
        x = block(x)
    ```

    Examples
    --------
    >>> cnn = ConvolutionalNeuralNetwork(3, [6, 6], 12).build()
    >>> x = torch.randn(3, 3, 32, 32)
    >>> y = cnn(x)
    >>> y.shape
    torch.Size([3, 12, 26, 26])
    
    """
    
    # Arguments.
    in_channels: int
    hidden_channels: List[int]
    out_channels: int
    out_activation: Type[nn.Module]

    # Attributes.
    blocks: Sequential[Conv2dBlock]

    def __init__( 
        self, 
        in_channels: int,
        hidden_channels: List[int],
        out_channels: int,
        out_activation: Type[nn.Module] = nn.ReLU,
    ) -> None:  
        super().__init__()

        blocks = Sequential[Conv2dBlock]()
        for in_ch, out_ch in zip([in_channels] + hidden_channels, 
                                 hidden_channels + [out_channels]):
            block = Conv2dBlock(in_ch, out_ch, kernel_size=3, padding=0, 
                                activation=nn.ReLU)
            blocks.append(block)
        
        # Set the activation function of the last block.
        blocks[-1].activated(out_activation)

    def forward(
        self, 
        x: torch.Tensor,  
    ) -> torch.Tensor:
        """Forward pass of the convolutional neural network.

        Evaluates the convolutional blocks sequentially.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor of shape (N, C, H, W).
            Where N is the batch size, C is the number of channels, H is the 
            height, and W is the width.
        
        Returns
        -------
        torch.Tensor
            The output tensor of shape (N, out_channels, H', W').
            Where N is the batch size, out_channels is the number of output 
            channels, H is the height, and W is the width.
        
        """
        
        for block in self.blocks:
            x = block(x)
        return x

### 4. Define Properties

There are some properties that should be defined in a component:
- `input`: The input block of the component.
- `output`: The output block of the component.
- `hidden`: The hidden blocks of the component (all except output).
These should generally be defined before the constructor, but after the annotations.

In the current example, this corresponds to the code:
```python
    @property
    def input(self) -> Conv2dBlock:
        return self.blocks[0]
    
    @property
    def output(self) -> Conv2dBlock:
        return self.blocks[-1]
    
    @property
    def hidden(self) -> ReferringLayerList[Conv2dBlock]:
        return self.blocks[:-1]
```

**NOTE:** If you subclass another component, it is likely that these will already be defined.

In [4]:
from deeplay.list import ReferringLayerList

class ConvolutionalNeuralNetwork(DeeplayModule):
    """A fully convolutional neural network.

    Parameters
    ----------
    in_channels : int
        The number of input channels.
    hidden_channels : List[int]
        The number of hidden channels.
    out_channels : int
        The number of output channels.
    out_activation : Type[nn.Module]
        The type of activation function of the output layer.
    
    Attributes
    ----------
    blocks : Sequential[Conv2dBlock]
        The list of convolutional blocks.
    
    Input
    -----
    x : torch.Tensor
        The input tensor of shape (N, C, H, W).
        Where N is the batch size, C is the number of channels, H is the 
        height, and W is the width.
        Additial dimensions before C are allowed.
    
    Output
    ------
    y : torch.Tensor
        The output tensor of shape (N, out_channels, H', W').
        Where N is the batch size, out_channels is the number of output 
        channels, H is the height, and W is the width.
        Additial dimensions before out_channels will be preserved.

    Evaluation
    ----------
    ```python
    for block in blocks:
        x = block(x)
    ```

    Examples
    --------
    >>> cnn = ConvolutionalNeuralNetwork(3, [6, 6], 12).build()
    >>> x = torch.randn(3, 3, 32, 32)
    >>> y = cnn(x)
    >>> y.shape
    torch.Size([3, 12, 26, 26])
    
    """
    
    # Arguments.
    in_channels: int
    hidden_channels: List[int]
    out_channels: int
    out_activation: Type[nn.Module]

    # Attributes.
    blocks: Sequential[Conv2dBlock]
    
    @property
    def input(self) -> Conv2dBlock:
        return self.blocks[0]
    
    @property
    def output(self) -> Conv2dBlock:
        return self.blocks[-1]
    
    @property
    def hidden(self) -> ReferringLayerList[Conv2dBlock]:
        return self.blocks[:-1]

    def __init__( 
        self, 
        in_channels: int,
        hidden_channels: List[int],
        out_channels: int,
        out_activation: Type[nn.Module] = nn.ReLU,
    ) -> None: 
        super().__init__()

        blocks = Sequential[Conv2dBlock]()
        for in_ch, out_ch in zip([in_channels] + hidden_channels, 
                                 hidden_channels + [out_channels]):
            block = Conv2dBlock(in_ch, out_ch, kernel_size=3, padding=0, 
                                activation=nn.ReLU)
            blocks.append(block)
        
        # Set the activation function of the last block.
        blocks[-1].activated(out_activation)

    def forward(
        self, 
        x: torch.Tensor,  
    ) -> torch.Tensor:
        """Forward pass of the convolutional neural network.

        Evaluates the convolutional blocks sequentially.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor of shape (N, C, H, W).
            Where N is the batch size, C is the number of channels, H is the 
            height, and W is the width.
        
        Returns
        -------
        torch.Tensor
            The output tensor of shape (N, out_channels, H', W').
            Where N is the batch size, out_channels is the number of output 
            channels, H is the height, and W is the width.
        
        """
        
        for block in self.blocks:
            x = block(x)
        return x

### 5. Implement Auxiliary Methods

It might make sense to add additional auxiliary methods to the component. These should generally be convenience methods for complex configurations. For example, a 
`ConvolutionalNeuralNetwork` could have a `pooled` method that adds pooling layers
to each block. 

In [5]:
from deeplay.external.layer import Layer
from typing_extensions import Self

class ConvolutionalNeuralNetwork(DeeplayModule):
    """A fully convolutional neural network.

    Parameters
    ----------
    in_channels : int
        The number of input channels.
    hidden_channels : List[int]
        The number of hidden channels.
    out_channels : int
        The number of output channels.
    out_activation : Type[nn.Module]
        The type of activation function of the output layer.
    
    Attributes
    ----------
    blocks : Sequential[Conv2dBlock]
        The list of convolutional blocks.
    
    Input
    -----
    x : torch.Tensor
        The input tensor of shape (N, C, H, W).
        Where N is the batch size, C is the number of channels, H is the 
        height, and W is the width.
        Additial dimensions before C are allowed.
    
    Output
    ------
    y : torch.Tensor
        The output tensor of shape (N, out_channels, H', W').
        Where N is the batch size, out_channels is the number of output 
        channels, H is the height, and W is the width.
        Additial dimensions before out_channels will be preserved.

    Evaluation
    ----------
    ```python
    for block in blocks:
        x = block(x)
    ```

    Examples
    --------
    >>> cnn = ConvolutionalNeuralNetwork(3, [6, 6], 12).build()
    >>> x = torch.randn(3, 3, 32, 32)
    >>> y = cnn(x)
    >>> y.shape
    torch.Size([3, 12, 26, 26])
    
    """
    
    # Arguments.
    in_channels: int
    hidden_channels: List[int]
    out_channels: int
    out_activation: Type[nn.Module]

    # Attributes.
    blocks: Sequential[Conv2dBlock]
    
    @property
    def input(self) -> Conv2dBlock:
        return self.blocks[0]
    
    @property
    def output(self) -> Conv2dBlock:
        return self.blocks[-1]
    
    @property
    def hidden(self) -> ReferringLayerList[Conv2dBlock]:
        return self.blocks[:-1]

    def __init__( 
        self, 
        in_channels: int,
        hidden_channels: List[int],
        out_channels: int,
        out_activation: Type[nn.Module] = nn.ReLU,
    ) -> None:  
        super().__init__()

        blocks = Sequential[Conv2dBlock]()
        for in_ch, out_ch in zip([in_channels] + hidden_channels, 
                                 hidden_channels + [out_channels]):
            block = Conv2dBlock(in_ch, out_ch, kernel_size=3, padding=0, 
                                activation=nn.ReLU)
            blocks.append(block)
        
        # Set the activation function of the last block.
        blocks[-1].activated(out_activation)

    def forward(
        self, 
        x: torch.Tensor,  
    ) -> torch.Tensor:
        """Forward pass of the convolutional neural network.

        Evaluates the convolutional blocks sequentially.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor of shape (N, C, H, W).
            Where N is the batch size, C is the number of channels, H is the 
            height, and W is the width.
        
        Returns
        -------
        torch.Tensor
            The output tensor of shape (N, out_channels, H', W').
            Where N is the batch size, out_channels is the number of output 
            channels, H is the height, and W is the width.
        
        """
        
        for block in self.blocks:
            x = block(x)
        return x

    def pooled(self, 
               pool: Layer = Layer(nn.MaxPool2d, 2),
               apply_to_first: bool = False,
               apply_to_last: bool = True) -> Self:
        """Add pooling layers after each block.

        Parameters
        ----------
        pool : Layer
            The pooling layer.
        apply_to_first : bool
            Whether to apply pooling to the first block.
        apply_to_last : bool
            Whether to apply pooling to the last block.
        
        """
        
        if apply_to_first:
            self.input.pooled(pool)
        if apply_to_last:
            self.output.pooled(pool)

        for block in self.hidden[1:]:
            block.pooled(pool)

        return self